# Subsystems enrichment analysis

Performs the subsystems enrichment analysis based on the hypergeometric test:
$$P(x \geq k) = 1 - hypergeom.cdf(k-1, M, n, N)$$

* k: number of diferentially expressed reactions in a subsystem,
* n: number of diferentially expressed reactions in the model,
* N: number of reactions in a subsystem,
* M: number of reactions in the model.

In [100]:
import pandas as pd
import numpy as np

from scipy.stats import hypergeom
#import statsmodels.stats.multitest as multi
from helpers import bh

import cobra


## Basic settings

In [101]:
require_biomass = False
folder_enrich = "enrichment\\biomass" if require_biomass else "enrichment\\no_biomass"

## Subsystems data
Get the subsystems data from the model.

In [102]:
#model = cobra.io.read_sbml_model('models\\Recon3D.xml')

In [103]:
#model_mat = cobra.io.load_matlab_model('models\\Recon3D.mat')

In [104]:
#reactions_subsystems = {}
#for r in model_mat.reactions:
#    reactions_subsystems[r.id] = r.subsystem

In [105]:
#df_subsystems = pd.DataFrame()
#df_subsystems['subsystem'] = reactions_subsystems.values()
#df_subsystems['reaction'] = reactions_subsystems.keys()
#df_subsystems.to_csv('models\\subsystems.csv', index=False)

In [106]:
df_subs = pd.read_csv('models\\subsystems.csv')
subsystems = df_subs.subsystem.unique()
df_subs.head()

,subsystem,reaction
0,"Transport, mitochondrial",24_25DHVITD3tm
1,"Transport, extracellular",25HVITD3t
2,"Transport, lysosomal",COAtl
3,Extracellular exchange,EX_5adtststerone_e
4,Extracellular exchange,EX_5adtststerones_e


## Reaction data

In [107]:
df_reactions = pd.read_csv(f"{folder_enrich}\\reactions.csv")

## Hypergeometric test
Go through the results of the reactions' enrichment analyses and use those to perform the subsystems' enrichment analysis.

In [108]:
df_enrichment = pd.DataFrame(columns=["subsystem", "p_up", "p_down", "q_up", "q_down", "enrichment", "p_changed", "q_changed", "changed"])
df_enrichment["subsystem"] = subsystems

M = len(df_reactions) # number of different reactions in pairs of models
n_up = sum(df_reactions.enrichment == 1) # number of upregulated reactions in models
n_down = sum(df_reactions.enrichment == -1)  # number of downregulated reactions in models
n_changed = sum(df_reactions.changed == 1)  # number of changed reactions in models

for subsystem in subsystems:
    subsystem_reactions = df_subsystems.loc[df_subsystems.subsystem == subsystem,'reaction'].values
    df_sub = df_reactions[df_reactions['reaction'].isin(subsystem_reactions)]
        
    #if not take_all:
    # option 1: take only remaining reactions
    N = len(df_sub) # number of reactions in a subsystem
    #else:
    #    # option 2: take all reactions from the original model
    #    N = len(df_subs[df_subs.subsystem == subsystem])
    k_up = sum(df_sub.enrichment == 1)# number of upregulated reactions in a subsystem
    k_down = sum(df_sub.enrichment == -1)# number of downregulated reactions in a subsystem
    k_changed = sum(df_sub.changed == 1)# number of changed reactions in a subsystem
    
    if n_up:         
        p_up = 1 - hypergeom.cdf(k_up-1, M, n_up, N)                
    else:
        p_up = 1.0
        
    if n_down:         
        p_down = 1 - hypergeom.cdf(k_down-1, M, n_down, N)                
    else:
        p_down = 1.0
        
    if n_changed:
        p_changed = 1 - hypergeom.cdf(k_changed, M, n_changed, N)                
    else:
        p_changed = 1
        
    df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_up'] = p_up
    df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_down'] = p_down
    df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_changed'] = p_changed
    

    
df_enrichment['q_up'] = bh(df_enrichment['p_up'])
df_enrichment['q_down'] = bh(df_enrichment['p_down'])
df_enrichment['q_changed'] = bh(df_enrichment['p_changed'])

    
df_enrichment.loc[(df_enrichment['q_up']<0.05) & (df_enrichment['q_up']<df_enrichment['q_down']),'enrichment'] = 1
df_enrichment.loc[(df_enrichment['q_down']<0.05) & (df_enrichment['q_down']<=df_enrichment['q_up']),'enrichment'] = -1
df_enrichment.loc[(df_enrichment['q_changed']<0.05),'changed'] = 1

df_enrichment = df_enrichment.fillna(0)




In [109]:
df_enrichment

,subsystem,p_up,p_down,q_up,q_down,enrichment,p_changed,q_changed,changed
0,"Transport, mitochondrial",0.999468,2.682781e-06,1.0,9.479159e-05,-1,0.000000e+00,0.000000e+00,1
1,"Transport, extracellular",0.999644,1.797906e-11,1.0,1.905781e-09,-1,2.960965e-13,4.023875e-13,1
2,"Transport, lysosomal",0.853889,5.393489e-01,1.0,1.000000e+00,0,8.895290e-01,1.000000e+00,0
3,Extracellular exchange,0.999969,2.413045e-08,1.0,1.278914e-06,-1,1.268992e-03,1.681415e-03,1
4,Vitamin D metabolism,1.000000,2.315444e-02,1.0,2.231246e-01,0,0.000000e+00,0.000000e+00,1
...,...,...,...,...,...,...,...,...,...
101,N-glycan metabolism,1.000000,1.000000e+00,1.0,1.000000e+00,0,0.000000e+00,0.000000e+00,1
102,Drug metabolism,0.906168,2.046172e-02,1.0,2.231246e-01,0,0.000000e+00,0.000000e+00,1
103,Protein formation,1.000000,2.853369e-01,1.0,1.000000e+00,0,0.000000e+00,0.000000e+00,1
104,Vitamin B12 metabolism,1.000000,1.000000e+00,1.0,1.000000e+00,0,0.000000e+00,0.000000e+00,1


In [110]:
df_enrichment.to_csv(f"{folder_enrich}\\subsystems.csv", index=False)